**IELE_4014**  
**Felipe Velásquez Montoya**
&nbsp;&nbsp;&nbsp;&nbsp; *cód estudiante:* 201632422
# Reto 3
Problema de clasificación de Pulsares utilizando el dataset HTRU2 (https://archive.ics.uci.edu/ml/datasets/HTRU2) y una regresión logística de desarrollo propio.

**Paso 0**   Instalanción de librería numpy para facilitar operaciones con matrices

In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


Importación de dependencias random, math y numpy.

In [3]:
#Se importa numpy para facilitar las operaciones con matrices
import random
import math
import numpy as np

# Desarrollo de la regresión logística

Se muestra, a continuación, el código de la regresión logística y sus respectivas funciones auxiliares. se inicia con la función train_test_split, diseñada para imitar el comportamiento de su homónima en sklearn. 

Este método divide un conjunto de datos en datos de entrenamiento y datos de prueba. Lo anterior se hace de manera aleatoria, por lo que no hay garantía alguna de que los conjuntos resultantes tengan la misma distribución de probabilidad.

La metodología utilizada consiste en recorrer las listas y generar un número al azar por cada elemento, si el número generado es menor o igual a la rata dada por el tamaño del conjunto de entrenamiento dividido entre el tamaño total del conjunto, se añade al conjunto de entrenamiento. De modo contrario, se añade al conjunto de pruebas.

In [4]:
#Método auxiliar que divide los datos entre entranamiento y test
#test_ratio es el porcentaje total de los datos que se usará para test.
#Hace la división al azar utilizando random.random
#Pre: X_set  y y_set deben tener el mismo orden, es decir, para todo elemento X_i 
# de la X_set, y_i en el conjunto y_set debe ser su respectiva pareja.
def train_test_split(X_set, y_set, test_ratio):
    
    test_size = math.floor(test_ratio*len(X_set))
    current_test_size = 0
        
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for i in range(len(X_set)):
        
        if random.random() <= test_ratio and current_test_size < test_size:
            X_test.append(X_set[i])
            y_test.append(y_set[i])
            current_test_size += 1
        else:
            X_train.append(X_set[i])
            y_train.append(y_set[i])
    
    #En caso de que test no quede del tamaño requerido, se sacan los últimos elementos de train
    while len(X_test) != test_size:
        X_test.append(X_train.pop())
        y_test.append(y_train.pop())
    
    return X_train, X_test, y_train, y_test
            

Se presenta, a continuación, la clase LogisticRegression. Esta contiene la función *fit* que utiliza un descenso de gradiente estocástico para entrenar el modelo. Las funciones *sigmoid* y *predict* sirven de funciones auxiliares y *accuracy* y *confusion_matrix* sirven para evaluar el modelo desarrollado.

*accuracy* da el número de aciertos (TP+TN) dividido entre el total de elementos. <br>
*confusion_matrix* calcula los elementos de la matriz de confusión del modelo, es decir, verdaderos positivos (TP), falsos positivos (FP), verdaderos negativos (TN) y falsos negativos (FN). 

In [5]:
class LogisticRegression:
    
    def __init__(self, sensitivity = 0.7, suppress_output = False):
        self.W = None
        self.sensitivity = sensitivity
        self.suppress_output = suppress_output
        
    #Descenso estocástico de gradiente para regresión lineal.
    #training_data_X es una matriz que representa el conjunto de x_i's
    #training_data_y es una matriz que representa el conjunto de y_i's. Esta matriz tiene únicamente una 
    #Pre: training_data_X  y training_data_y deben tener el mismo orden, es decir, para todo elemento X_i 
    # de la matriz training_data_X, y_i en el conjunto training_data_y debe ser su respectiva pareja.
    # ejemplo, la fila 0 de training_data_X tiene como respectivo valor y el que está dado por 
    # la fila 0 de training_data_y
    #Pre: training_data_y es 0 o 1.
    def fit(self, training_data_X, training_data_y,learning_rate= 0.0001, max_iterations = 10000):
        
        #Para evitar que la embarre y ponga un learning rate demasiado alto que lleve a que el método no converja 
        if learning_rate > 0.0001:
            learning_rate = 0.0001
    
        #Se inicializa W en ceros.
        W = np.zeros(shape = (1, len(training_data_X[0])))
    
        iterations = 0
        no_improv_it = 0
        last_error = 0
        new_error = 0
    
        #Se estimará el gradiente a partir, únicamente, de uno de los datos
        while iterations < max_iterations and no_improv_it < 100 :
        
            #Seleccionar un dato al azar.
            i = math.floor(random.random()*len(training_data_X))
    
            #Calcular error del dato seleccionado al azar.
            e = training_data_y[i] - self.predict(W, training_data_X[i])
            
            #Estimar gradiente con el error anterior
            estimated_grad = np.multiply(e, training_data_X[i])
        
            W = W + np.multiply(learning_rate, estimated_grad)
            
            #Si no hubo una mejora en el error cuadrático, reducir tasa de aprendizaje (puede que se esté "saltando" el punto óptimo)
            if last_error < e:
                learning_rate*=0.9
                no_improv_it = 0
            elif last_error == e:
                no_improv_it += 1
            else:
                no_improv_it = 0
        
            last_error = e
            iterations += 1        
        
        self.W = W
        
        if not self.suppress_output:
            print("----------------------------------------------")
            print("SDG finished, finishing error %s" % last_error)
            print("W found: %s" % W)
            print(" Score found for the model with training data: %s" % self.accuracy(training_data_X, training_data_y))
           

        
    #Calcula el error.
    def accuracy(self, test_data_X, test_data_y):
    
        correct = 0
        incorrect = 0
        
        for i in range(0, len(test_data_X)):
        
            #si se predice 1 pero es 0
            if  self.predict(self.W, test_data_X[i]) >= self.sensitivity and test_data_y[i] == 0:
                incorrect += 1
            #si se predice 1 y es 1
            elif self.predict(self.W, test_data_X[i]) >= self.sensitivity and test_data_y[i] == 1:
                correct+=1
            #si se predice 0 y es 1
            elif self.predict(self.W, test_data_X[i]) < self.sensitivity and test_data_y[i] == 1:
                incorrect+=1
            #si se predice 0 y es 0
            else:
                correct+=1
                
        return correct/(correct+incorrect)

    def sigmoid(self, x):
        if x < 0:
            return 1 - 1/(1+math.exp(x))
        else:
            return 1/(1+math.exp(-x))

    def predict(self, W, X_i):
        return self.sigmoid(np.dot(X_i, W.transpose())[0])
    
    def confusion_matrix(self, test_data_X, test_data_y):
        
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        
        for i in range(0, len(test_data_X)):
            
            #si se predice 1 pero es 0
            if  self.predict(self.W, test_data_X[i]) >= self.sensitivity and test_data_y[i] == 0:
                fp += 1
            #si se predice 1 y es 1
            elif self.predict(self.W, test_data_X[i]) >= self.sensitivity and test_data_y[i] == 1:
                tp += 1
            #si se predice 0 y es 1
            elif self.predict(self.W, test_data_X[i]) < self.sensitivity and test_data_y[i] == 1:
                fn += 1
            #si se predice 0 y es 0
            else:
                tn += 1
                
        return tp, fp, tn, fn

# Solución del problema de clasificación de pulsares
Ya expuesta la regresión logística que se desarrolló, se probará la misma atacando el problema de clasificación de pulsares

In [6]:
data_matrix = np.loadtxt(open("./HTRU2/HTRU_2.csv", "rb"), delimiter=",", skiprows=0)

print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

X = np.resize(data_matrix, (len(data_matrix), len(data_matrix[0])-1))
y = data_matrix[:,len(data_matrix[0])-1]

Filas de la matriz: 17898
Columnas de la matriz: 9


A continuación, se usa el método *train_test_split* desarrollado para dividir los datos en dos conjuntos, datos de prueba con 10000 elementos y datos de entrenamiento, con los 7898 restantes.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_ratio = 10000.00/float(len(X))) 

print("X_train size: %s" %len(X_train))
print("y_train size: %s" %len(y_train)) 
print("X_test size: %s" %len(X_test)) 
print("y_test size: %s" %len(y_test)) 

X_train size: 7898
y_train size: 7898
X_test size: 10000
y_test size: 10000


Se vuelve a utilizar el método *train_test_split*, esta vez para dividir los datos de entrenamiento en conjuntos de 100, 500, 1000 y 5000 datos. Como se puede comprobar por la información impresa en consola, son creados cuatro nuevos conjuntos de entrenamiento.

In [8]:
sizes = [100.00, 500.00, 1000.00, 5000.00]
X_train_array = []
y_train_array = []

for i in sizes:
    Xy = train_test_split(X_train, y_train, test_ratio = 1.00 - i/float(len(X_train)))
    X_train_array.append(Xy[0])
    y_train_array.append(Xy[2])
    
#Verificar tamaños de los nuevos sets de entrenamiento
for i in range(0, len(X_train_array)):
    print("-----------------------------")
    print("Set de entrenamiento %s" % i)
    print("    Tamaño X de entrenamiento: %s | Tamaño y de entrenamiento: %s" 
          % (len(X_train_array[i]), len(y_train_array[i])))

-----------------------------
Set de entrenamiento 0
    Tamaño X de entrenamiento: 100 | Tamaño y de entrenamiento: 100
-----------------------------
Set de entrenamiento 1
    Tamaño X de entrenamiento: 500 | Tamaño y de entrenamiento: 500
-----------------------------
Set de entrenamiento 2
    Tamaño X de entrenamiento: 1000 | Tamaño y de entrenamiento: 1000
-----------------------------
Set de entrenamiento 3
    Tamaño X de entrenamiento: 5000 | Tamaño y de entrenamiento: 5000


Ahora se evaluan los modelos con los datos de entrenamiento.

In [9]:
logreg_array = []
logreg_score = []

for i in range(0, len(X_train_array)):
    
    lg = LogisticRegression(suppress_output = True)
    
    #Se entrena la regresión logística con el conjunto de entrenamiento correspondiente.
    logreg_array.append(lg)
    lg.fit(X_train_array[i], y_train_array[i])
    
    #Se prueba la regresión logística con los conjuntos de prueba:
    logreg_score.append(logreg_array[i].accuracy(X_test, y_test))
    
    #Se imprimen los resultados del modelo
    print("------------------------------------------------------------------------")
    print("El Modelo entrenado con %s datos predijo correctamente el %s porciento de los datos de entrenamiento" 
         % (sizes[i], logreg_score[i]))
    print("Matriz de confusión encontrada (TP, FP, TN, FN): %s" % str(lg.confusion_matrix(X_test,y_test)))

------------------------------------------------------------------------
El Modelo entrenado con 100.0 datos predijo correctamente el 0.9057 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (0, 0, 9057, 943)
------------------------------------------------------------------------
El Modelo entrenado con 500.0 datos predijo correctamente el 0.9057 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (0, 0, 9057, 943)
------------------------------------------------------------------------
El Modelo entrenado con 1000.0 datos predijo correctamente el 0.9057 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (0, 0, 9057, 943)
------------------------------------------------------------------------
El Modelo entrenado con 5000.0 datos predijo correctamente el 0.9057 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (0, 0, 9057, 943)


Obsérvese que al igual que en el reto 1, la regresión logística predice que todos los datos hacen parte de la clase mayoritaria. Por esto mismo, se observa que la exactitud del modelo es igual al porcentaje de datos pertenecientes a la clase mayoritaria.

En este caso, sin embargo, es posible ajustar la *sensibilidad* (porcentaje de seguridad a partir del cual se dice que el dato pertenece a la clase 1). En la regresión implementada, toma un valor por defecto de 0.7, sin embargo, si se reduce a 0.25 podrá verse que, a pesar de que la exactitud de los modelos se reduce, esta vez clasifican a algunos objetos (así sea erróneamente) como pertenecientes a la clase 1. <br>

In [10]:
logreg_array = []
logreg_score = []

for i in range(0, len(X_train_array)):
    
    lg = LogisticRegression(suppress_output = True, sensitivity = 0.25)
    
    #Se entrena la regresión logística con el conjunto de entrenamiento correspondiente.
    logreg_array.append(lg)
    lg.fit(X_train_array[i], y_train_array[i])
    
    #Se prueba la regresión logística con los conjuntos de prueba:
    logreg_score.append(logreg_array[i].accuracy(X_test, y_test))
    
    #Se imprimen los resultados del modelo
    print("------------------------------------------------------------------------")
    print("El Modelo entrenado con %s datos predijo correctamente el %s porciento de los datos de entrenamiento" 
         % (sizes[i], logreg_score[i]))
    print("Matriz de confusión encontrada (TP, FP, TN, FN): %s" % str(lg.confusion_matrix(X_test,y_test)))

------------------------------------------------------------------------
El Modelo entrenado con 100.0 datos predijo correctamente el 0.8317 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (87, 827, 8230, 856)
------------------------------------------------------------------------
El Modelo entrenado con 500.0 datos predijo correctamente el 0.8243 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (97, 911, 8146, 846)
------------------------------------------------------------------------
El Modelo entrenado con 1000.0 datos predijo correctamente el 0.8755 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (33, 335, 8722, 910)
------------------------------------------------------------------------
El Modelo entrenado con 5000.0 datos predijo correctamente el 0.8074 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (115, 1098, 795

Como puede observarse, la exactitud del modelo se no se ve mayormente afectada y, ahora, algunos de los datos se predicen como pertenecientes a la clase 1. Sin embargo, debe resaltarse que aún hay un gran número de falsos negativos. Por lo tanto, si lo que se buscara hacer con este modelo fuera un programa que ayudara a los científicos a "marcar" los objetos candidatos a ser pulsares, este no sería adecuado, pues no marcaría a la mayoría de lo objetos correspondientes a pulsares. 

Al igual que en el Reto1, se realizó un preprocesamiento con SMOTE en la herramienta Knime para ver si, estando los datos balanceados, se podría entrenar un modelo que obtuviera un buen puntaje en los datos de prueba.

In [12]:
data_matrix_smote = np.loadtxt(open("./HTRU2/HTRU_2_oversampled.csv", "rb"), delimiter=",", skiprows=0)

print("Filas de la matriz: " + str(len(data_matrix_smote)))
print("Columnas de la matriz: " + str(len(data_matrix_smote[0])))

X_smote = np.resize(data_matrix_smote, (len(data_matrix_smote), len(data_matrix_smote[0])-1))
y_smote = data_matrix_smote[:,len(data_matrix_smote[0])-1]

X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote, y_smote, test_ratio = 10000.00/float(len(X_smote))) 

print("Tamaño X entrenamiento: %s | Tamaño y entrenamiento: %s" % (len(X_train_smote), len(y_train_smote)))
print("Tamaño X prueba: %s | Tamaño y prueba: %s" % (len(X_test_smote), len(y_test_smote)))

sizes_smote = [100.00, 500.00, 1000.00, 5000.00]
X_train_array_smote = []
y_train_array_smote = []

for i in sizes_smote:
    Xy_smote = train_test_split(X_train_smote, y_train_smote, test_ratio = 1.00 - i/float(len(X_train_smote)))
    X_train_array_smote.append(Xy_smote[0])
    y_train_array_smote.append(Xy_smote[2])
    
#Verificar tamaños de los nuevos sets de entrenamiento
for i in range(0, len(X_train_array_smote)):
    print("-----------------------------")
    print("Set de entrenamiento %s" % i)
    print("    Tamaño X de entrenamiento: %s | Tamaño y de entrenamiento: %s" 
          % (len(X_train_array_smote[i]), len(y_train_array_smote[i])))
    
logreg_array_smote = []
logreg_score_smote = []

for i in range(0, len(X_train_array_smote)):
    
    lg = LogisticRegression(suppress_output = True, sensitivity = 0.5)
    
    #Se entrena la regresión logística con el conjunto de entrenamiento correspondiente.
    logreg_array_smote.append(lg)
    lg.fit(X_train_array_smote[i], y_train_array_smote[i])
    
    #Se prueba la regresión logística con los conjuntos de prueba:
    logreg_score_smote.append(logreg_array_smote[i].accuracy(X_test, y_test))
    
    #Se imprimen los resultados del modelo
    print("------------------------------------------------------------------------")
    print("El Modelo entrenado con %s datos predijo correctamente el %s porciento de los datos de entrenamiento" 
         % (sizes[i], logreg_score_smote[i]))
    print("Matriz de confusión encontrada (TP, FP, TN, FN): %s" % str(lg.confusion_matrix(X_test,y_test)))


Filas de la matriz: 32518
Columnas de la matriz: 9
Tamaño X entrenamiento: 22518 | Tamaño y entrenamiento: 22518
Tamaño X prueba: 10000 | Tamaño y prueba: 10000
-----------------------------
Set de entrenamiento 0
    Tamaño X de entrenamiento: 100 | Tamaño y de entrenamiento: 100
-----------------------------
Set de entrenamiento 1
    Tamaño X de entrenamiento: 500 | Tamaño y de entrenamiento: 500
-----------------------------
Set de entrenamiento 2
    Tamaño X de entrenamiento: 1000 | Tamaño y de entrenamiento: 1000
-----------------------------
Set de entrenamiento 3
    Tamaño X de entrenamiento: 5000 | Tamaño y de entrenamiento: 5000
------------------------------------------------------------------------
El Modelo entrenado con 100.0 datos predijo correctamente el 0.7053 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (198, 2202, 6855, 745)
------------------------------------------------------------------------
El Modelo entrenado con 5

Como puede observarse, se obtienen resultados inconsistentes y en varios casos pobres, cada ejecución de este último bloque de código genera modelos con rendimiento diferente.

Vale la pena también mencionar que se realizaron pruebas en las que se normalizaban los datos, sin embargo, no se encontró mayor diferencia en el desempeño de los modelos.

Se improta sklearn para utilizar su función de proprocesamiento StrandardScaler

In [14]:
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [13]:
from sklearn.preprocessing import StandardScaler

data_matrix_norm = np.loadtxt(open("./HTRU2/HTRU_2.csv", "rb"), delimiter=",", skiprows=0)

print("Filas de la matriz: " + str(len(data_matrix_norm )))
print("Columnas de la matriz: " + str(len(data_matrix_norm [0])))

X_norm = np.resize(data_matrix_norm, (len(data_matrix_norm), len(data_matrix_norm[0])-1))
y_norm = data_matrix_norm[:,len(data_matrix_norm[0])-1]

X_norm = StandardScaler().fit_transform(X_norm)

X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_ratio = 10000.00/float(len(X_norm))) 

print("Tamaño X entrenamiento: %s | Tamaño y entrenamiento: %s" % (len(X_train_norm), len(y_train_norm)))
print("Tamaño X prueba: %s | Tamaño y prueba: %s" % (len(X_test_norm), len(y_test_norm)))

sizes_norm = [100.00, 500.00, 1000.00, 5000.00]
X_train_array_norm = []
y_train_array_norm = []

for i in sizes_smote:
    Xy_norm = train_test_split(X_train_norm, y_train_norm, test_ratio = 1.00 - i/float(len(X_train_norm)))
    X_train_array_norm.append(Xy_norm[0])
    y_train_array_norm.append(Xy_norm[2])
    
#Verificar tamaños de los nuevos sets de entrenamiento
for i in range(0, len(X_train_array_norm)):
    print("-----------------------------")
    print("Set de entrenamiento %s" % i)
    print("    Tamaño X de entrenamiento: %s | Tamaño y de entrenamiento: %s" 
          % (len(X_train_array_norm[i]), len(y_train_array_norm[i])))
    
logreg_array_norm = []
logreg_score_norm = []

for i in range(0, len(X_train_array_norm)):
    
    lg = LogisticRegression(suppress_output = True, sensitivity = 0.5)
    
    #Se entrena la regresión logística con el conjunto de entrenamiento correspondiente.
    logreg_array_norm.append(lg)
    lg.fit(X_train_array_norm[i], y_train_array_norm[i])
    
    #Se prueba la regresión logística con los conjuntos de prueba:
    logreg_score_norm.append(logreg_array_norm[i].accuracy(X_test_norm, y_test_norm))
    
    #Se imprimen los resultados del modelo
    print("------------------------------------------------------------------------")
    print("El Modelo entrenado con %s datos predijo correctamente el %s porciento de los datos de entrenamiento" 
         % (sizes[i], logreg_score_norm[i]))
    print("Matriz de confusión encontrada (TP, FP, TN, FN): %s" % str(lg.confusion_matrix(X_test_norm,y_test_norm)))


Filas de la matriz: 17898
Columnas de la matriz: 9
Tamaño X entrenamiento: 7898 | Tamaño y entrenamiento: 7898
Tamaño X prueba: 10000 | Tamaño y prueba: 10000
-----------------------------
Set de entrenamiento 0
    Tamaño X de entrenamiento: 100 | Tamaño y de entrenamiento: 100
-----------------------------
Set de entrenamiento 1
    Tamaño X de entrenamiento: 500 | Tamaño y de entrenamiento: 500
-----------------------------
Set de entrenamiento 2
    Tamaño X de entrenamiento: 1000 | Tamaño y de entrenamiento: 1000
-----------------------------
Set de entrenamiento 3
    Tamaño X de entrenamiento: 5000 | Tamaño y de entrenamiento: 5000
------------------------------------------------------------------------
El Modelo entrenado con 100.0 datos predijo correctamente el 0.517 porciento de los datos de entrenamiento
Matriz de confusión encontrada (TP, FP, TN, FN): (411, 4308, 4759, 522)
------------------------------------------------------------------------
El Modelo entrenado con 500.

Como se observa, inclusive con la estandarización de los datos, obtiene un rendimiento bajo con un alto número de falsos positivos.